In [2]:
// //package stackoverflow
// import org.apache.spark.SparkContext
// import org.apache.spark.SparkContext._
// import org.apache.spark.SparkConf
// import org.apache.spark.rdd.RDD
// import scala.xml._

In [3]:
// val sc = new SparkContext()

org.apache.spark.SparkException: A master URL must be set in your configuration

In [121]:
import scala.xml.{ NodeSeq, MetaData }
import java.io.File
import scala.io.{ BufferedSource, Source }

abstract class StackTable[T] {


  def getDate(n: scala.xml.NodeSeq): Long = n.text match {
    case "" => 0
    case s => dateFormat.parse(s).getTime
  }

  def dateFormat = {
    import java.text.SimpleDateFormat
    import java.util.TimeZone
    val f = new SimpleDateFormat("yyyy-MM-dd'T'HH:mm:ss.SSS")
    f.setTimeZone(TimeZone.getTimeZone("GMT"))
    f
  }

  def getInt(n: scala.xml.NodeSeq): Int = n.text match {
    case "" => 0
    case x => x.toInt
  }

  def parseXml(x: scala.xml.Elem): T

  def parse(s: String): Option[T] =
    if (s.startsWith("  <row ")) Some(parseXml(scala.xml.XML.loadString(s)))
    else None
}

object Post extends StackTable[Post] {

//   val file = new File("data/Posts.xml")
//   assert(file.exists)

  override def parseXml(x: scala.xml.Elem): Post = Post(
    getInt(x \ "@Id"),
    getInt(x \ "@PostTypeId"),
    getInt(x \ "@AcceptedAnswerId"),
    getDate(x \ "@CreationDate"),
    getInt(x \ "@Score"),
    getInt(x \ "@ViewCount"),
    (x \ "@Body").text,
    getInt(x \ "@OwnerUserId"),
    getDate(x \ "@LastActivityDate"),
    (x \ "@Title").text,
    getTags(x \ "@Tags"),
    getInt(x \ "@AnswerCount"),
    getInt(x \ "@CommentCount"),
    getInt(x \ "@FavoriteCount"),
    getDate(x \ "@CommunityOwnedDate"))

  def getTags(x: scala.xml.NodeSeq): Array[String] = x.text match {
    case "" => Array()
    case s => s.drop(1).dropRight(1).split("><")
  }
}

// <row Id="1" PostTypeId="1" AcceptedAnswerId="15" CreationDate="2010-07-19T19:12:12.510" Score="19" ViewCount="1033" Body="&lt;p&gt;How should I elicit prior distributions from experts when fitting a Bayesian model?&lt;/p&gt;&#xA;" OwnerUserId="8" LastActivityDate="2010-09-15T21:08:26.077" Title="Eliciting priors from experts" Tags="&lt;bayesian&gt;&lt;prior&gt;&lt;elicitation&gt;" AnswerCount="5" CommentCount="1" FavoriteCount="11" />
case class Post(
  id: Int,
  postTypeId: Int,
  acceptedAnswerId: Int,
  creationDate: Long,
  score: Int,
  viewCount: Int,
  body: String,
  ownerUserId: Int,
  lastActivityDate: Long,
  title: String,
  tags: Array[String],
  answerCount: Int,
  commentCount: Int,
  favoriteCount: Int,
  communityOwnedDate: Long) 

object User extends StackTable[User] {

//   val file = new File("data/Users.xml")
//   assert(file.exists)

  override def parseXml(x: scala.xml.Elem): User = User(
    getInt(x \ "@Id"),
    getInt(x \ "@Reputation"),
    getDate(x \ "@CreationDate"),
    (x \ "@DisplayName").text,
    getDate(x \ "@LastAccessDate"),
    (x \ "@WebsiteUrl").text,
    (x \ "@Location").text,
    (x \ "@AboutMe").text,
    getInt(x \ "@Views"),
    getInt(x \ "@UpVotes"),
    getInt(x \ "@DownVotes"),
    (x \ "@EmailHash").text,
    getInt(x \ "@Age"))
}

// <row Id="1" Reputation="5828" CreationDate="2009-04-30T07:08:27.067" DisplayName="Jeff Atwood" LastAccessDate="2013-08-15T00:13:25.560" WebsiteUrl="http://www.codinghorror.com/blog/" Location="El Cerrito, CA" AboutMe="&lt;p&gt;&lt;a href=&quot;http://www.codinghorror.com/blog/archives/001169.html&quot; rel=&quot;nofollow&quot;&gt;Stack Overflow Valued Associate #00001&lt;/a&gt;&lt;/p&gt;&#xA;&#xA;&lt;p&gt;Wondering how our software development process works? &lt;a href=&quot;http://www.youtube.com/watch?v=08xQLGWTSag&quot; rel=&quot;nofollow&quot;&gt;Take a look!&lt;/a&gt;&lt;/p&gt;&#xA;&#xA;&lt;p&gt;Find me &lt;a href=&quot;http://twitter.com/codinghorror&quot; rel=&quot;nofollow&quot;&gt;on twitter&lt;/a&gt;, or &lt;a href=&quot;http://www.codinghorror.com/blog&quot; rel=&quot;nofollow&quot;&gt;read my blog&lt;/a&gt;. Don't say I didn't warn you &lt;em&gt;because I totally did&lt;/em&gt;.&lt;/p&gt;&#xA;&#xA;&lt;p&gt;However, &lt;a href=&quot;http://www.codinghorror.com/blog/2012/02/farewell-stack-exchange.html&quot; rel=&quot;nofollow&quot;&gt;I no longer work at Stack Exchange, Inc&lt;/a&gt;. I'll miss you all. Well, &lt;em&gt;some&lt;/em&gt; of you, anyway. :)&lt;/p&gt;&#xA;" 
// Views="6020" UpVotes="2130" DownVotes="32" EmailHash="51d623f33f8b83095db84ff35e15dbe8" Age="43" />
case class User(
  id: Int,
  reputation: Int,
  creationDate: Long,
  displayName: String,
  lastAccessDate: Long,
  websiteUrl: String,
  location: String,
  aboutMe: String,
  views: Int,
  upVotes: Int,
  downVotes: Int,
  emailHash: String,
  age: Int) 

object Vote extends StackTable[Vote] {

  override def parseXml(x: scala.xml.Elem): Vote = Vote(
    getInt(x \ "@Id"),
    getInt(x \ "@PostId"),
    getInt(x \ "@VoteTypeId"),
    getInt(x \ "@UserId"),
    getDate(x \ "@CreationDate"))
}

// <row Id="1264793" PostId="486481" VoteTypeId="5" UserId="175880" CreationDate="2013-05-30T00:00:00.000" />
case class Vote(
  id: Int,
  postId: Int,
  voteTypeId: Int,
  userId: Int,
  creationDate: Long) 

In [6]:
val allPosts = spark.textFile("/home/vagrant/miniprojects/spark/data/spark-stats-data/allPosts/part-00*.xml.gz")

/home/vagrant/miniprojects/spark/data/spark-stats-data/allPosts/part-00*.xml.gz MapPartitionsRDD[3] at textFile at <console>:12

In [67]:
val allUsers = spark.textFile("/home/vagrant/miniprojects/spark/data/spark-stats-data/allUsers/part-00*.xml.gz")

/home/vagrant/miniprojects/spark/data/spark-stats-data/allUsers/part-00*.xml.gz MapPartitionsRDD[30] at textFile at <console>:27

In [71]:
val allVoters = spark.textFile("/home/vagrant/miniprojects/spark/data/spark-stats-data/allVotes/part-00*.xml.gz")

/home/vagrant/miniprojects/spark/data/spark-stats-data/allVotes/part-00*.xml.gz MapPartitionsRDD[35] at textFile at <console>:27

In [195]:
allPosts.flatMap(Post.parse).take(4)

org.apache.spark.SparkException: Job aborted due to stage failure: ClassNotFound with classloader: ClasspathFilter(
  parent = URLClassLoader with NativeCopyLoader with RawResources(
  urls = ArraySeq(/home/vagrant/.ivy2/cache/sh.jove/jove-bridge_2.10/jars/jove-bridge_2.10-0.1.1-1-SNAPSHOT.jar, /home/vagrant/.ivy2/cache/org.apache.spark/spark-repl_2.10/jars/spark-repl_2.10-1.3.0.jar, /home/vagrant/.ivy2/cache/org.scala-lang/jline/jars/jline-2.10.4.jar, /home/vagrant/.ivy2/cache/org.fusesource.jansi/jansi/jars/jansi-1.4.jar, /home/vagrant/.ivy2/cache/org.apache.spark/spark-core_2.10/jars/spark-core_2.10-1.3.0.jar, /home/vagrant/.ivy2/cache/com.twitter/chill_2.10/jars/chill_2.10-0.5.0.jar, /home/vagrant/.ivy2/cache/com.twitter/chill-java/jars/chill-java-0.5.0.jar, /home/vagrant/.ivy2/cache/com.esotericsoftware.kryo/kryo/bundles/kryo-2.21.jar, /home/vagrant/.ivy2/cache/com.esotericsoftware.reflectasm/reflectasm/jars/reflectasm-1.07-shaded.jar, /home/vagrant/.ivy2/cache/com.esotericsoftwar

In [33]:
val objData = allPosts.filter(line => line.startsWith("  <row "))
              .

Array("  <row AcceptedAnswerId="15" AnswerCount="5" Body="&lt;p&gt;How should I elicit prior distributions from experts when fitting a Bayesian model?&lt;/p&gt;&#10;" CommentCount="1" CreationDate="2010-07-19T19:12:12.510" FavoriteCount="17" Id="1" LastActivityDate="2010-09-15T21:08:26.077" OwnerUserId="8" PostTypeId="1" Score="26" Tags="&lt;bayesian&gt;&lt;prior&gt;&lt;elicitation&gt;" Title="Eliciting priors from experts" ViewCount="1457" />")

In [45]:
abstract class StackTable[T] {


  def getDate(n: scala.xml.NodeSeq): Long = n.text match {
    case "" => 0
    case s => dateFormat.parse(s).getTime
  }

  def dateFormat = {
    import java.text.SimpleDateFormat
    import java.util.TimeZone
    val f = new SimpleDateFormat("yyyy-MM-dd'T'HH:mm:ss.SSS")
    f.setTimeZone(TimeZone.getTimeZone("GMT"))
    f
  }

  def getInt(n: scala.xml.NodeSeq): Int = n.text match {
    case "" => 0
    case x => x.toInt
  }

  def parseXml(x: scala.xml.Elem): T

  def parse(s: String): Option[T] =
    if (s.startsWith("  <row ")) Some(parseXml(scala.xml.XML.loadString(s)))
    else None
}

In [ ]:
allPosts.map(
_ => (_.favoriteCount, (_.)))
.map({
    case ()
})

In [ ]:
val salesByCountry = (transactions.map(t => (t.userId, t.amount))
                .join(users.map(u => (u.userId, u.country)))
                .map({ 
                  case (userId: String, (amount: Int, country: String)) 
                      => (country, amount)
                }) 
                .reduceByKey(_ + _)
                .collect())

In [73]:
//allUsers.flatMap(User.parse).take(10)

: 

In [171]:
val top50 = allVoters.flatMap(Vote.parse).map({
    vote => (vote.postId, 
             (if (vote.voteTypeId == 5) 1 else 0, 
              if (vote.voteTypeId == 2) 1 else 0,
              if (vote.voteTypeId == 3) 1 else 0 ))
    // (1, (1, 0, 1))
}).reduceByKey(
    (x,y)=>(x._1 + y._1, x._2 + y._2, x._3 + y._3)
    //(1, (10, 7, 5))
).map({
    x => (x._2._1, (x._2._2, x._2._3))
    // (10, (7, 5))
}).reduceByKey(
    (x, y) => (x._1 + y._1, x._2 + y._2)
    // x (7,5) favoriate = 10
    // y (6,7) favoriate = 10
    // (10, (7+6, 5+7))
).sortByKey(true).take(50)

// .takeOrdered(50)(
//     Ordering[Int].reverse.on(x => x._1)
// ).map({
//     x => (x._1, x._2._1 / (x._2._1 + x._2._2))
// })

Array((0,(232141,11828)), (1,(23800,702)), (2,(13064,187)), (3,(7811,79)), (4,(5321,52)), (5,(3753,28)), (6,(2900,15)), (7,(2370,22)), (8,(1785,10)), (9,(1865,6)), (10,(1423,12)), (11,(1052,9)), (12,(1189,5)), (13,(733,2)), (14,(822,5)), (15,(842,6)), (16,(688,0)), (17,(474,0)), (18,(698,1)), (19,(468,1)), (20,(327,1)), (21,(533,3)), (22,(453,1)), (23,(415,2)), (24,(204,0)), (25,(347,0)), (26,(311,5)), (27,(360,4)), (28,(206,1)), (29,(367,1)), (30,(218,1)), (31,(329,2)), (32,(66,0)), (33,(221,0)), (34,(164,0)), (35,(82,0)), (36,(213,0)), (37,(110,1)), (38,(320,2)), (39,(121,1)), (40,(119,0)), (41,(51,0)), (42,(52,0)), (44,(76,0)), (45,(64,0)), (47,(181,0)), (48,(103,0)), (49,(148,0)), (50,(53,0)), (52,(114,0)))

In [172]:
top50.map({
    x => (x._1, x._2._1 / (x._2._1 + x._2._2).toDouble)
})

Array((0,0.9515184306202837), (1,0.971349277609991), (2,0.9858878575201871), (3,0.9899873257287706), (4,0.990321980271729), (5,0.9925945517058979), (6,0.9948542024013722), (7,0.9908026755852842), (8,0.9944289693593314), (9,0.9967931587386424), (10,0.9916376306620209), (11,0.9915174363807728), (12,0.9958123953098827), (13,0.9972789115646259), (14,0.9939540507859734), (15,0.9929245283018868), (16,1.0), (17,1.0), (18,0.9985693848354793), (19,0.997867803837953), (20,0.9969512195121951), (21,0.9944029850746269), (22,0.9977973568281938), (23,0.9952038369304557), (24,1.0), (25,1.0), (26,0.9841772151898734), (27,0.989010989010989), (28,0.9951690821256038), (29,0.9972826086956522), (30,0.9954337899543378), (31,0.9939577039274925), (32,1.0), (33,1.0), (34,1.0), (35,1.0), (36,1.0), (37,0.990990990990991), (38,0.9937888198757764), (39,0.9918032786885246), (40,1.0), (41,1.0), (42,1.0), (44,1.0), (45,1.0), (47,1.0), (48,1.0), (49,1.0), (50,1.0), (52,1.0))

In [183]:
for (pair <- top50) {
    val sum = 0
    sum = sum + pair._1
}

: 

In [194]:
var i = 0
var sum = 0.0
while (i < top50.length) {
    sum += top50(i)._1
    i += 1
}
println(sum/50.0)

24.76


In [185]:
top50(1)._1

1